In [ ]:
import pandas as pd
df = pd.read_csv('D:\SoB\MainSoB.csv',sep=",",header=0, index_col=False)
df = df.drop(df.columns[0], axis=1)

In [ ]:
df.head()

In [ ]:
#Drop Irrelevant Rows
print(len(df))
Territories_To_Drop = [2, 11, 15, 72, 78]
df = df[~df['State.Code'].isin(Territories_To_Drop)]
#Drops Alaska, DC, Hawaii, Puerto Rico and the Virgin Islands
print(len(df))
df = df[~(df['Commodity.Code']==9999)]
# Drop the "Other" Commodity
df = df[df['Quantity.Type'].str.strip()=='Acres']
# Drops Trees, Clams, Raisins, Non/Plant
print(len(df))

In [ ]:
# Function to add leading 0 to a string if its length is smaller than the maximum length in the column
def add_leading_zero(column):
    max_length = df[column].astype(str).str.len().max()
    return df[column].astype(str).apply( lambda string: string.zfill(max_length) if len(string) < max_length else string)

In [ ]:
#Create FIPS out of state and county components
df['FIPS']=add_leading_zero('State.Code')+add_leading_zero('County.Code')
df.tail()

In [ ]:
#ID is Year x County tuple. Use FIPS to get what we mean when we say County
df['ID']=df['Year'].astype(str)+'_'+df['FIPS']
print(df['ID'].nunique())
CommodityCombinations=df.groupby('ID')['Commodity.Code'].agg(frozenset).value_counts()
#Gets each combination of remaining commodity codes for initial inspection. 
#No immediate favorites. Will have missing values problem. But can minimize it?

In [ ]:
CommodityCombinations

In [ ]:
# Generate List of Overlapping Commodities

#import pandas as pd

# Step 1: Group the DataFrame by 'ID' and aggregate 'Commodity' as a set
grouped = df.groupby('ID')['Commodity.Code'].agg(set).reset_index()

# Step 2: Create a dictionary to store commodity frequency counts
commodity_counts = {}

# Step 3: Iterate over rows and update commodity count dictionary
for _, row in grouped.iterrows():
    commodities = row['Commodity.Code']
    for commodity in commodities:
        if commodity in commodity_counts:
            commodity_counts[commodity] += 1
        else:
            commodity_counts[commodity] = 1

# Step 4: Sort commodities based on frequency count in descending order
sorted_commodities = sorted(commodity_counts, key=commodity_counts.get, reverse=True)

# Step 5: Select commodities with shared IDs while limiting missing values
selected_commodities = []
selected_ids = set()

for commodity in sorted_commodities:
    # Get IDs for the current commodity
    ids = set(grouped[grouped['Commodity.Code'].apply(lambda x: commodity in x)]['ID'])
    
    # Find common IDs with previously selected commodities
    common_ids = ids.intersection(selected_ids)
    
    # Check if including this commodity increases the length of selected list
    if len(common_ids) < len(selected_commodities):
        continue
    
    # Update selected IDs and commodities
    selected_ids.update(ids)
    selected_commodities.append(commodity)

In [ ]:
#Will check the above against which commodities have significant total presence in the FCIP
#Generate list of most insured crops by total acres
totaled_df = df.groupby('Commodity.Code')['Net.Planted.Quantity'].sum().sort_values(ascending=False).reset_index()
totaled_df.to_csv('TotalInsuredQuantity.csv',index=True)

In [ ]:
OverlappingAlgorithmCommodities = selected_commodities[:20]
HighlyInsuredCommodities = totaled_df['Commodity.Code'].tolist()[:20]
temp = set(OverlappingAlgorithmCommodities+HighlyInsuredCommodities)# List of Commodity.Codes

# Set 'Commodity.Code' as the index and select 'Commodity.Name' column
code_name_df = df.set_index('Commodity.Code')['Commodity.Name']

# Create a dictionary using the code_name_df DataFrame
code_name_dict = code_name_df.to_dict()

# Filter the dictionary for the specified codes
filtered_dict = {code: code_name_dict.get(code) for code in temp}

In [ ]:
labels = [filtered_dict.get(item) for item in OverlappingAlgorithmCommodities]
OverlappingAlgorithmComms= pd.DataFrame(zip(OverlappingAlgorithmCommodities,labels), columns=['Values','Label'])

In [ ]:
OverlappingAlgorithmComms

In [ ]:
labels = [filtered_dict.get(item) for item in HighlyInsuredCommodities]
HighlyInsuredComms= pd.DataFrame(zip(HighlyInsuredCommodities,labels), columns=['Values','Label'])

In [ ]:
HighlyInsuredComms

In [ ]:
#Attach the Rank of most insured crops to the list of most overlapping crops
HighlyInsuredComms['Rank'] = HighlyInsuredComms.index+1
temp= pd.DataFrame(HighlyInsuredComms[['Values','Rank']])
merged_comms = OverlappingAlgorithmComms.merge(temp, on='Values', how='left')
merged_comms.rename(columns={'Values':'Commodity.Code','Label':'Commodity.Name','Rank':'Quantity.Insured.Rank'},inplace=True)
merged_comms

In [ ]:
#Based on these results, I suggest Corn, Wheat, Soybeans, Grain Sorghum, and Barley as commonly insured staples for our analysis
#Cotton and Rangeland should also be included, tentatively, due to high available overlap
values_list=[41,11,81,51,88,91,21]
Final=merged_comms[merged_comms['Commodity.Code'].isin(values_list)]
Final.to_csv('SelectedCommodities.csv',index=True)
Final

In [ ]:
# Counterarguments
#Why only within county? Could assign each county itself and neighboring counties in order to have fewer missing commodities, more interesting divergence
#Food for thought for future